# Workshop Notebook 2: Vetting a Model With Production Experiments

So far, we've discussed practices and methods for transitioning an ML model and related artifacts from development to production. However, just the act of pushing a model into production is not the only consideration. In many situations, it's important to vet a model's performance in the real world before fully activating it. Real world vetting can surface issues that may not have arisen during the development stage, when models are only checked using hold-out data.

In this notebook, you will learn about two kinds of production ML model validation methods: A/B testing and Shadow Deployments. A/B tests and other types of experimentation are part of the ML lifecycle. The ability to quickly experiment and test new models in the real world helps data scientists to continually learn, innovate, and improve AI-driven decision processes.

## Prerequisites

This notebook assumes that you have completed Notebook 1 "Deploy a Model", and that at this point you have:

* Created and worked with a Workspace.
* Uploaded ML Models and worked with Wallaroo Model Versions.
* Created a Wallaroo Pipeline, added model versions as pipeline steps, and deployed the pipeline.
* Performed sample inferences and undeployed the pipeline.

The same workspace, models, and pipelines will be used for this notebook.

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.


In [1]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from IPython.display import display
from IPython.display import Image
import pandas as pd
import json
import datetime
import time
import cv2
import matplotlib.pyplot as plt
import string
import random
import pyarrow as pa
import sys
import asyncio
pd.set_option('display.max_colwidth', None)

import sys
 
# setting path - only needed when running this from the `with-code` folder.
sys.path.append('../')

import utils

import time


In [2]:
## convenience functions from the previous notebook

# return the workspace called <name> through the Wallaroo client.
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
            return workspace
    # if no workspaces were found
    if workspace==None:
        raise KeyError(f"Workspace {name} was not found.")
    return workspace


# returns the most recent model version in a workspace for the matching `model_name`
def get_model_version(model_name, workspace):
    modellist = workspace.models()
    model_version = [m.versions()[-1] for m in modellist if m.name() == model_name]
    # if no models match, return None
    if len(modellist) <= 0:
        raise KeyError(f"Model {mname} not found in this workspace")
        return None
    return model_version[0]

# get a pipeline by name in the workspace
def get_pipeline(pipeline_name, workspace):
    plist = workspace.pipelines()
    pipeline = [p for p in plist if p.name() == pipeline_name]
    if len(pipeline) <= 0:
        raise KeyError(f"Pipeline {pipeline_name} not found in this workspace")
        return None
    return pipeline[0]


#### Pre-exercise

If needed, log into Wallaroo and go to the workspace, pipeline, and most recent model version from the ones that you created in the previous notebook. Please refer to Notebook 1 to refresh yourself on how to log in and set your working environment to the appropriate workspace.

In [3]:
## blank space to log in 

wl = wallaroo.Client()

# retrieve the previous workspace, model, and pipeline version

workspace_name = 'workshop-workspace-john-cv'

workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

model_name = 'mobilenet'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'cv-retail'

pipeline = wl.get_pipeline(pipeline_name)

# display the workspace, pipeline and model version
display(workspace)
display(pipeline)
display(prime_model_version)


{'name': 'workshop-workspace-john-cv', 'id': 6, 'archived': False, 'created_by': '93dc3434-cf57-47b4-a3e4-b2168b1fd7e1', 'created_at': '2023-12-08T18:10:29.076812+00:00', 'models': [{'name': 'mobilenet', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 12, 8, 18, 10, 33, 449382, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 12, 8, 18, 10, 33, 449382, tzinfo=tzutc())}, {'name': 'cv-post-process-drift-detection', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 12, 8, 18, 10, 33, 729844, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 12, 8, 18, 10, 33, 729844, tzinfo=tzutc())}], 'pipelines': [{'name': 'cv-retail', 'create_time': datetime.datetime(2023, 12, 8, 18, 10, 33, 817785, tzinfo=tzutc()), 'definition': '[]'}]}

name,cv-retail
created,2023-12-08 18:10:33.817785+00:00
last_updated,2023-12-08 18:10:34.108308+00:00
deployed,False
arch,None
tags,
versions,"e7149fb9-8253-4e87-b9a6-a220c8df5639, c6abcdaa-047b-4eef-8f83-eb2dcc8782d1"
steps,mobilenet
published,False


Name,mobilenet
Version,57373e66-6dd7-426a-87ea-85de61ce7386
File Name,mobilenet.pt.onnx
SHA,9044c970ee061cc47e0c77e20b05e884be37f2a20aa9c0c3ce1993dbd486a830
Status,ready
Image Path,None
Architecture,None
Updated At,2023-08-Dec 18:10:33


## Upload Challenger Models

Multiple models can be uploaded to a Wallaroo workspace and deployed in a Pipeline as a `pipeline step`.  These can include pre or post processing Python steps, models that take in different input and output types, or ML models that are of totally different frameworks.

For this module, we will are focuses on different models that have the same input and outputs that are compared to each other to find the "best" model to use.  Before we start, we'll upload another set of models that were pre-trained to provide house prices.  In 'Deploy a Model' we used the file `xgb_model.onnx` as our model.

For this exercise, we will ignore the post-processing step so we can show the various examples without it.

### Upload Challenger Models Exercise

Upload to the workspace set in the steps above the challenger models.  The following is an example of uploading the Resnet50 model.

Because image size may vary from one image to the next, converting the image to a tensor array may have a different shape from one image to the next.  For example, a 640x480 image produces an array of `[640][480][3]` for 640 rows with 480 columns each, and each pixel has 3 possible color values.

Because the tensor array size may change from image to image, the model upload sets the model's batch configuration to `batch_config="single"`.  See the Wallaroo [Data Schema Definitions](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-developer-helper-guides/wallaroo-data-schema-guide/) for more details.

```python
cv_resnet_version = (wl.upload_model('resnet50',
                                    '../models/frcnn-resnet.pt.onnx',
                                    framework=wallaroo.framework.Framework.ONNX)
                                    .configure(tensor_fields=["tensor"],
                                               batch_config="single"
                                               )
                    )
```

In [5]:
cv_resnet_version = (wl.upload_model('resnet50',
                                    '../models/frcnn-resnet.pt.onnx',
                                    framework=wallaroo.framework.Framework.ONNX)
                                    .configure(tensor_fields=["tensor"],
                                               batch_config="single"
                                               )
                    )

In [6]:
display(cv_resnet_version)

Name,resnet50
Version,d8712e05-c1a4-4b15-b313-dacb570f4379
File Name,frcnn-resnet.pt.onnx
SHA,43326e50af639105c81372346fb9ddf453fea0fe46648b2053c375360d9c1647
Status,ready
Image Path,None
Architecture,None
Updated At,2023-08-Dec 18:12:08


## A/B Pipeline Steps

An [A/B test](https://en.wikipedia.org/wiki/A/B_testing), also called a controlled experiment or a randomized control trial, is a statistical method of determining which of a set of variants is the best. A/B tests allow organizations and policy-makers to make smarter, data-driven decisions that are less dependent on guesswork.

In the simplest version of an A/B test, subjects are randomly assigned to either the **_control group_** (group A) or the **_treatment group_** (group B). Subjects in the treatment group receive the treatment (such as a new medicine, a special offer, or a new web page design) while the control group proceeds as normal without the treatment. Data is then collected on the outcomes and used to study the effects of the treatment.

In data science, A/B tests are often used to choose between two or more candidate models in production, by measuring which model performs best in the real world. In this formulation, the control is often an existing model that is currently in production, sometimes called the **_champion_**. The treatment is a new model being considered to replace the old one. This new model is sometimes called the **_challenger_**. In our discussion, we'll use the terms *champion* and *challenger*, rather than *control* and *treatment*.

When data is sent to a Wallaroo A/B test pipeline for inference, each datum is randomly sent to either the champion or challenger. After enough data has been sent to collect statistics on all the models in the A/B test pipeline, then those outcomes can be analyzed to determine the difference (if any) in the performance of the champion and challenger. Usually, the purpose of an A/B test is to decide whether or not to replace the champion with the challenger.

Keep in mind that in machine learning, the terms experiments and trials also often refer to the process of finding a training configuration that works best for the problem at hand (this is sometimes called hyperparameter optimization). In this guide, we will use the term experiment to refer to the use of A/B tests to compare the performance of different models in production.

There are a number of considerations to designing an A/B test; you can check out the article [*The What, Why, and How of A/B Testing*](https://wallarooai.medium.com/the-what-why-and-how-of-a-b-testing-64471847cd7e) for more details. In these exercises, we will concentrate on the deployment aspects.  You will need a champion model and  at least one challenger model. You also need to decide on a data split: for example 50-50 between the champion and challenger, or a 2:1 ratio between champion and challenger (two-thirds of the data to the champion, one-third to the challenger).

As an example of creating an A/B test deployment, suppose you have a champion model called "champion", that you have been running in a one-step pipeline called "pipeline". You now want to compare it to a challenger model called "challenger". For your A/B test, you will send two-thirds of the data to the champion, and the other third to the challenger. Both models have already been uploaded.

A/B pipeline steps are created with one of the two following commands:

* `wallaroo.pipeline.add_random_split([(weight1, model1), (weight2, model2)...])`: Create a new A/B Pipeline Step with the provided models and weights.
* `wallaroo.pipeline.replace_with_random_split(index, [(weight1, model1), (weight2, model2)...])`: Replace an existing Pipeline step with an A/B pipeline step at the specified index.

For A/B testing, pipeline steps are **added** or **replace** an existing step.

To **add** a A/B testing step use the Pipeline `add_random_split` method with the following parameters:

| Parameter | Type | Description |
| --- | --- | ---|
| **champion_weight** | Float (Required) | The weight for the champion model. |
| **champion_model** | Wallaroo.Model (Required) | The uploaded champion model. |
| **challenger_weight** | Float (Required) | The weight of the challenger model. |
| **challenger_model** | Wallaroo.Model (Required) | The uploaded challenger model. |
| **hash_key** | String(Optional) | A key used instead of a random number for model selection.  This must be between 0.0 and 1.0. |


Note that multiple challenger models with different weights can be added as the random split step.

In this example, a pipeline will be built with a 2:1 weighted ratio between the champion and a single challenger model.

```python
pipeline.add_random_split([(2, control), (1, challenger)]))
```

To **replace** an existing pipeline step with an A/B testing step use the Pipeline `replace_with_random_split` method.

| Parameter | Type | Description |
| --- | --- | ---|
| **index** | Integer (Required) | The pipeline step being replaced. |
| **champion_weight** | Float (Required) | The weight for the champion model. |
| **champion_model** | Wallaroo.Model (Required) | The uploaded champion model. |
| **challenger_weight** | Float (Required) | The weight of the challenger model. |
| **challenger_model** | Wallaroo.Model (Required) | The uploaded challenger model. |
| **hash_key** | String(Optional) | A key used instead of a random number for model selection.  This must be between 0.0 and 1.0. |

This example replaces the first pipeline step with a 2:1 champion to challenger radio.

```python
pipeline.replace_with_random_split(0,[(2, control), (1, challenger)]))
```

In either case, the random split will randomly send inference data to one model based on the weighted ratio.  As more inferences are performed, the ratio between the champion and challengers will align more and more to the ratio specified.

Reference:  [Wallaroo SDK Essentials Guide: Pipeline Management A/B Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipeline/#ab-testing).

Each model receives inputs that are approximately proportional to the weight it is assigned. For example, with two models having weights 1 and 1, each will receive roughly equal amounts of inference inputs. If the weights were changed to 1 and 2, the models would receive roughly 33% and 66% respectively instead.

When choosing the model to use, a random number between 0.0 and 1.0 is generated. The weighted inputs are mapped to that range, and the random input is then used to select the model to use. For example, for the two-models equal-weight case, a random key of 0.4 would route to the first model, 0.6 would route to the second.

Models used for A/B pipeline steps should have the **same** inputs and outputs to accurately compare each other.

Reference:  [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipeline/).

### A/B Pipeline Steps Exercise

Create an A/B pipeline step with the uploaded model versions using the `wallaroo.pipeline.add_random_split` method.  Since we have 3 models, apply a 2:1:1 ratio to the control:challenger1:challenger2 models.  For this exercise, we will not use the post processing model we uploaded in the previous notebook.

Since this pipeline was used in the previous notebook, use the `wallaroo.pipeline.clear()` method to clear all of the previous steps before adding the new one.  Recall that pipeline steps are not saved from the Notebook to the Wallaroo instance until the `wallaroo.pipeline.deploy(deployment_configuration)` method is applied.

One done, deploy the pipeline with `deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()`.

Here's an example of adding our A/B pipeline step and deploying it.

```python
my_pipeline.clear()
my_pipeline.add_random_split([(2, prime_model_version), (1, house_price_rf_model_version), (1, house_price_gbr_model_version)])
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
my_pipeline.deploy(deployment_config=deploy_config)
```

In [12]:
pipeline.undeploy()
pipeline.clear()
pipeline.add_random_split([(2, prime_model_version), (1, cv_resnet_version)])

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(2).cpus(1).memory("2Gi").build()
pipeline.deploy(deployment_config=deploy_config)

Waiting for undeployment - this will take up to 45s .................................... ok
Waiting for deployment - this will take up to 45s ........ ok


name,cv-retail
created,2023-12-08 18:10:33.817785+00:00
last_updated,2023-12-08 18:13:22.698167+00:00
deployed,True
arch,None
tags,
versions,"7ce5dd31-5851-492b-ba36-80e4adf8ef04, 0a4ce878-3f59-4134-b32f-1a6946db9775, e7149fb9-8253-4e87-b9a6-a220c8df5639, c6abcdaa-047b-4eef-8f83-eb2dcc8782d1"
steps,mobilenet
published,False


The pipeline steps are displayed with the Pipeline `steps()` method.  This is used to verify the current **deployed** steps in the pipeline.

* **IMPORTANT NOTE**: Verify that the pipeline is deployed before checking for pipeline steps.  Deploying the pipeline sets the steps into the Wallaroo system - until that happens, the steps only exist in the local system as *potential* steps.

In [8]:
# blank space to get the current pipeline steps

pipeline.steps()

[{'RandomSplit': {'hash_key': None, 'weights': [{'model': {'name': 'mobilenet', 'version': '57373e66-6dd7-426a-87ea-85de61ce7386', 'sha': '9044c970ee061cc47e0c77e20b05e884be37f2a20aa9c0c3ce1993dbd486a830'}, 'weight': 2}, {'model': {'name': 'resnet50', 'version': 'd8712e05-c1a4-4b15-b313-dacb570f4379', 'sha': '43326e50af639105c81372346fb9ddf453fea0fe46648b2053c375360d9c1647'}, 'weight': 1}]}}]

## A/B Pipeline Inferences

Inferences through pipelines that have A/B steps is the same any other pipeline:  use either `wallaroo.pipeline.infer(pandas record|Apache Arrow)` or `wallaroo.pipeline.infer_from_file(path)`.  The distinction is that inference data will randomly be assigned to one of the models in the A/B pipeline step based on the weighted ratio pattens.

The output is the same as any other inference request, with one difference:  the `out._model_split` field that lists which model version was used for the model split step.  Here's an example:

```python
single_result = pipeline.infer_from_file('./data/singleton.df.json')
display(single_result)
```

|&nbsp;|out._model_split|out.variable|
|---|---|---|
|0|[{"name":"house-price-prime","version":"69f344e6-2ab5-47e8-82a6-030e328c8d35","sha":"31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c"}]|[2176827.0]|

Please note that for batch inferences, the entire batch will be sent to the same model. So in order to verify that your pipeline is distributing inferences in the proportion you specified, you will need to send your queries one datum at a time.  This example has 4,000 rows of data submitted in one batch as an inference request.  Note that the same model is listed in the `out._model_split` field.

```python
multiple_result = pipeline.infer_from_file('../data/test_data.df.json')
display(multiple_result.head(10).loc[:, ['out._model_split', 'out.variable']])
```

|&nbsp;|out._model_split|out.variable|
|---|---|---|
|0|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[718013.7]|
|1|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[615094.6]|
|2|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[448627.8]|
|3|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[758714.3]|
|4|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[513264.66]|
|5|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[668287.94]|
|6|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[1004846.56]|
|7|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[684577.25]|
|8|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[727898.25]|
|9|[{"name":"house-price-rf-model","version":"616c2306-bf93-417b-9656-37bee6f14379","sha":"e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6"}]|[559631.06]|

To help with the next exercise, here is another convenience function you might find useful.  It takes the inference result and returns the model version used in the A/B Testing step.

Run the cell block below before going on to the exercise.

In [9]:
# get the names of the inferring models
# from a dataframe of a/b test results
def get_names(resultframe):
    modelcol = resultframe['out._model_split']
    jsonstrs = [mod[0]  for mod in modelcol]
    return [json.loads(jstr)['name'] for jstr in jsonstrs]

### A/B Pipeline Inferences Exercise

Perform a set of inferences with the same data, and show the model version used for the A/B Testing step.  Here's an example.

```python
for x in range(10):
    single_result = my_pipeline.infer_from_file('./data/singleton.df.json')
    display("{single_result.loc[0, 'out.variable']}")
    display(get_names(single_result))
```

In [10]:
width, height = 640, 480
dfImage, resizedImage = utils.loadImageAndConvertToDataframe('../data/images/example/dairy_bottles.png', width, height)

In [14]:
for x in range(10):
    single_result = pipeline.infer(dfImage, timeout=300)
    display(get_names(single_result))


['resnet50']

['mobilenet']

['mobilenet']

['resnet50']

['mobilenet']

['mobilenet']

['mobilenet']

['mobilenet']

['mobilenet']

['mobilenet']

## Shadow Deployments

Another way to vet your new model is to set it up in a shadow deployment. With shadow deployments, all the models in the experiment pipeline get all the data, and all inferences are recorded. However, the pipeline returns only one "official" prediction: the one from default, or champion model.

Shadow deployments are useful for "sanity checking" a model before it goes truly live. For example, you might have built a smaller, leaner version of an existing model using knowledge distillation or other model optimization techniques, as discussed [here](https://wallaroo.ai/how-to-accelerate-computer-vision-model-inference/). A shadow deployment of the new model alongside the original model can help ensure that the new model meets desired accuracy and performance requirements before it's put into production.

As an example of creating a shadow deployment, suppose you have a champion model called "champion", that you have been running in a one-step pipeline called "pipeline". You now want to put a challenger model called "challenger" into a shadow deployment with the champion. Both models have already been uploaded. 

Shadow deployments can be **added** as a pipeline step, or **replace** an existing pipeline step.

Shadow deployment steps are added with the `add_shadow_deploy(champion, [model2, model3,...])` method, where the `champion` is the model that the inference results will be returned.  The array of models listed after are the models where inference data is also submitted with their results displayed as as shadow inference results.

Shadow deployment steps replace an existing pipeline step with the  `replace_with_shadow_deploy(index, champion, [model2, model3,...])` method.  The `index` is the step being replaced with pipeline steps starting at 0, and the `champion` is the model that the inference results will be returned.  The array of models listed after are the models where inference data is also submitted with their results displayed as as shadow inference results.

Then creating a shadow deployment from a previously created (and deployed) pipeline could look something like this:

```python
# retrieve handles to the most recent versions 
# of the champion and challenger models
# see the A/B test section for the definition of get_model()
champion = get_model("champion")
challenger = get_model("challenger")

# get the existing pipeline and undeploy it
# see the A/B test section for the definition of get_pipeline()
pipeline = get_pipeline("pipeline")
pipeline.undeploy()

# clear the pipeline and add a shadow deploy step
pipeline.clear()
pipeline.add_shadow_deploy(champion, [challenger])
pipeline.deploy()
```

The above code clears the pipeline and adds a shadow deployment. The pipeline will still only return the inferences from the champion model, but it will also run the challenger model in parallel and log the inferences, so that you can compare what all the models do on the same inputs.

You can add multiple challengers to a shadow deploy:

```python
pipeline.add_shadow_deploy(champion, [challenger01, challenger02])
```

You can also create a shadow deployment from scratch with a new pipeline.  This example just uses two models - one champion, one challenger.

```python
newpipeline = wl.build_pipeline("pipeline")
newpipeline.add_shadow_deploy(champion, [challenger])
```

### Shadow Deployments Exercise

Use the champion and challenger models that you created in the previous exercises to create a shadow deployment. You can either create one from scratch, or reconfigure an existing pipeline.

At the end of this exercise, you should have a shadow deployment running multiple models in parallel.

Here's an example:

```python
pipeline.undeploy()

pipeline.clear()
pipeline.add_shadow_deploy(prime_model_version, 
                           [house_price_rf_model_version, 
                            house_price_gbr_model_version]
                        )

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)
```

In [15]:
# blank space to create a shadow deployment

pipeline.undeploy()

pipeline.clear()
pipeline.add_shadow_deploy(prime_model_version, 
                           [cv_resnet_version]
                        )

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("2Gi").build()
pipeline.deploy(deployment_config=deploy_config)

Waiting for undeployment - this will take up to 45s .......................................... ok
Waiting for deployment - this will take up to 45s ........ ok


name,cv-retail
created,2023-12-08 18:10:33.817785+00:00
last_updated,2023-12-08 18:15:02.176981+00:00
deployed,True
arch,None
tags,
versions,"43ef6552-12c1-4d4c-a8d0-58a0209a721f, 7ce5dd31-5851-492b-ba36-80e4adf8ef04, 0a4ce878-3f59-4134-b32f-1a6946db9775, e7149fb9-8253-4e87-b9a6-a220c8df5639, c6abcdaa-047b-4eef-8f83-eb2dcc8782d1"
steps,mobilenet
published,False


## Shadow Deploy Inference

Since a shadow deployment returns multiple predictions for a single datum, its inference result will look a little different from those of an A/B test or a single-step pipeline. The next exercise will show you how to examine all the inferences from all the models.

Model outputs are listed by column based on the model’s outputs. The output data is set by the term out, followed by the name of the model. For the default model, this is out.{variable_name}, while the shadow deployed models are in the format out_{model name}.variable, where {model name} is the name of the shadow deployed model.

Here's an example with the models `ccfraudrf` and `ccfraudxgb`.

```python
sample_data_file = './smoke_test.df.json'
response = pipeline.infer_from_file(sample_data_file)
```

| | time | in.tensor | out.dense_1 | check_failures | out_ccfraudrf.variable | out_ccfraudxgb.variable
|---|---|---|---|---|---|---
|0 | 2023-03-03 17:35:28.859 | [1.0678324729, 0.2177810266, -1.7115145262, 0.682285721, 1.0138553067, -0.4335000013, 0.7395859437, -0.2882839595, -0.447262688, 0.5146124988, 0.3791316964, 0.5190619748, -0.4904593222, 1.1656456469, -0.9776307444, -0.6322198963, -0.6891477694, 0.1783317857, 0.1397992467, -0.3554220649, 0.4394217877, 1.4588397512, -0.3886829615, 0.4353492889, 1.7420053483, -0.4434654615, -0.1515747891, -0.2668451725, -1.4549617756] | [0.0014974177] | 0 | [1.0] | [0.0005066991]

### Shadow Deploy Inference Exercise

Use the test data that from the previous exercise to send a single datum to the shadow deployment that you created in the previous exercise.  View the outputs from each of the shadow deployed models.

```python
single_result = pipeline.infer(dfImage)
display(single_result.columns)
display(single_result.loc[:, ['time', 'out.confidences', 'out_resnet50.confidences']])
```

In [16]:
# blank space to send an inference and examine the result

single_result = pipeline.infer(dfImage)
display(single_result.columns)
display(single_result.loc[:, ['time', 'out.confidences', 'out_resnet50.confidences']])

Index(['time', 'in.tensor', 'out.boxes', 'out.classes', 'out.confidences',
       'check_failures', 'out_resnet50.boxes', 'out_resnet50.classes',
       'out_resnet50.confidences'],
      dtype='object')

,time,out.confidences,out_resnet50.confidences
0,2023-12-08 18:15:11.692,"[0.98649, 0.90115356, 0.6077846, 0.5922323, 0.53729033, 0.4513168, 0.43728516, 0.43094054, 0.4084834, 0.39185277, 0.35759133, 0.3181266, 0.26451287, 0.23062895, 0.20482065, 0.174621, 0.17313862, 0.15999581, 0.14913696, 0.1366402, 0.13322707, 0.12218643, 0.121301256, 0.11956108, 0.11527827, 0.09616333, 0.08654833, 0.078406945, 0.07234089, 0.062820904, 0.052787986]","[0.9965358, 0.9883404, 0.9700247, 0.9696426, 0.96478045, 0.96037567, 0.9542889, 0.9467539, 0.946524, 0.94484967, 0.93611854, 0.91653466, 0.9133634, 0.8874814, 0.84405905, 0.825526, 0.82326967, 0.81740034, 0.7956525, 0.78669065, 0.7731486, 0.75193685, 0.7360918, 0.7009186, 0.6932351, 0.65077204, 0.63243586, 0.57877576, 0.5023476, 0.50163734, 0.44628552, 0.42804396, 0.4253787, 0.39086252, 0.36836442, 0.3473236, 0.32950658, 0.3105372, 0.29076362, 0.28558296, 0.26680034, 0.26302803, 0.25444376, 0.24568668, 0.2353662, 0.23321979, 0.22612995, 0.22483191, 0.22332378, 0.21442991, 0.20122288, 0.19754867, 0.19439234, 0.19083925, 0.1871393, 0.17646024, 0.16628945, 0.16326219, 0.14825206, 0.13694529, 0.12920643, 0.12815322, 0.122357555, 0.121289656, 0.116281696, 0.11498632, 0.111848116, 0.11016138, 0.1095062, 0.1039151, 0.10385688, 0.097573474, 0.09632071, 0.09557622, 0.091599144, 0.09062039, 0.08262358, 0.08223499, 0.07993951, 0.07989185, 0.078758545, 0.078201495, 0.07737936, 0.07690251, 0.07593444, 0.07503418, 0.07482597, 0.068981, 0.06841128, 0.06764157, 0.065750405, 0.064908616, 0.061884128, 0.06010121, 0.0578873, 0.05717648, 0.056616478, 0.056017116, 0.05458274, 0.053669468]"


## After the Experiment: Swapping in New Models

You have seen two methods to validate models in production with test (challenger) models. 
The end result of an experiment is a decision about which model becomes the new champion. Let's say that you have been running the shadow deployment that you created in the previous exercise,  and you have decided that you want to replace the model "champion" with the model "challenger". To do this, you will clear all the steps out of the pipeline, and add only "challenger" back in.

```python
# retrieve a handle to the challenger model
# see the A/B test section for the definition of get_model()
challenger = get_model("challenger")

# get the existing pipeline and undeploy it
# see the A/B test section for the definition of get_pipeline()
pipeline = get_pipeline("pipeline")
pipeline.undeploy()

# clear out all the steps and add the champion back in 
pipeline.clear() 
pipeline.add_model_step(challenger).deploy()
```

### After the Experiment: Swapping in New Models Exercise

Pick one of your challenger models as the new champion, and reconfigure your shadow deployment back into a single-step pipeline with the new chosen model.

* Run the test datum from the previous exercise through the reconfigured pipeline.
* Compare the results to the results from the previous exercise.
* Notice that the pipeline predictions are different from the old champion, and consistent with the new one.

At the end of this exercise, you should have a single step pipeline, running a new model.

In [17]:
# Blank space - remove all steps, then redeploy with new champion model
pipeline.undeploy()
pipeline.clear()

pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("2Gi").build()
pipeline.deploy(deployment_config=deploy_config)

single_result = pipeline.infer(dfImage)
display(single_result)
display(pipeline.steps())

Waiting for undeployment - this will take up to 45s .................................... ok
Waiting for deployment - this will take up to 45s ............. ok


,time,in.tensor,out.boxes,out.classes,out.confidences,check_failures
0,2023-12-08 18:16:07.004,"[0.9372549057, 0.9529411793, 0.9490196109, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9490196109, 0.9490196109, 0.9529411793, 0.9529411793, 0.9490196109, 0.9607843161, 0.9686274529, 0.9647058845, 0.9686274529, 0.9647058845, 0.9568627477, 0.9607843161, 0.9647058845, 0.9647058845, 0.9607843161, 0.9647058845, 0.9725490212, 0.9568627477, 0.9607843161, 0.9176470637, 0.9568627477, 0.9176470637, 0.8784313798, 0.8941176534, 0.8431372643, 0.8784313798, 0.8627451062, 0.850980401, 0.9254902005, 0.8470588326, 0.9686274529, 0.8941176534, 0.8196078539, 0.850980401, 0.9294117689, 0.8666666746, 0.8784313798, 0.8666666746, 0.9647058845, 0.9764705896, 0.980392158, 0.9764705896, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.980392158, 0.8941176534, 0.4823529422, 0.4627451003, 0.4313725531, 0.270588249, 0.2588235438, 0.2941176593, 0.3450980484, 0.3686274588, 0.4117647111, 0.4549019635, 0.4862745106, 0.5254902244, 0.5607843399, 0.6039215922, 0.6470588446, 0.6862745285, 0.721568644, 0.7450980544, 0.7490196228, 0.7882353067, 0.8666666746, 0.980392158, 0.9882352948, 0.9686274529, 0.9647058845, 0.9686274529, 0.9725490212, 0.9647058845, 0.9607843161, 0.9607843161, 0.9607843161, 0.9607843161, ...]","[0.0, 210.2901, 85.26463, 479.07495, 72.03781, 197.3227, 151.44223, 468.43225, 211.28015, 184.72838, 277.2192, 420.42746, 143.23904, 203.83005, 216.85547, 448.8881, 13.095016, 41.91339, 640.0, 480.0, 106.51464, 206.14499, 159.54643, 463.96756, 278.0637, 1.5217237, 321.45782, 93.563484, 462.31833, 104.16202, 510.5396, 224.75314, 310.4559, 1.3959818, 352.8513, 94.123825, 528.0485, 268.4225, 636.26715, 475.7666, 220.06293, 0.51385117, 258.31833, 90.18019, 552.87115, 96.30235, 600.7256, 233.53384, 349.24072, 0.27034378, 404.17325, 98.68022, 450.89346, 264.2356, 619.6033, 472.65173, 261.51385, 193.4335, 307.17914, 408.75247, 509.22018, 101.16539, 544.1857, 235.7374, 592.4824, 100.38687, 633.77985, 239.13432, 475.54208, 297.6141, 551.0544, 468.01547, 368.81982, 163.61407, 423.90942, 362.7888, 120.669, 0.0, 175.9362, 81.774574, 72.48429, 0.0, 143.50789, 85.4698, 271.12686, 200.89185, 305.626, 274.59537, 161.80728, 0.0, 213.08308, 85.42828, 162.13324, 0.0, 214.60814, 83.81444, 310.89108, 190.95468, 367.32925, 397.28137, ...]","[44, 44, 44, 44, 82, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 84, 84, 44, 84, 44, 44, 44, 61, 44, 86, 44, 44]","[0.98649, 0.90115356, 0.6077846, 0.5922323, 0.53729033, 0.4513168, 0.43728516, 0.43094054, 0.4084834, 0.39185277, 0.35759133, 0.3181266, 0.26451287, 0.23062895, 0.20482065, 0.174621, 0.17313862, 0.15999581, 0.14913696, 0.1366402, 0.13322707, 0.12218643, 0.121301256, 0.11956108, 0.11527827, 0.09616333, 0.08654833, 0.078406945, 0.07234089, 0.062820904, 0.052787986]",0


[{'ModelInference': {'models': [{'name': 'mobilenet', 'version': '57373e66-6dd7-426a-87ea-85de61ce7386', 'sha': '9044c970ee061cc47e0c77e20b05e884be37f2a20aa9c0c3ce1993dbd486a830'}]}}]

In [18]:
# change the step without undeploying

pipeline.clear()

pipeline.add_model_step(cv_resnet_version)

pipeline.deploy(deployment_config=deploy_config)

# provide time for the swap to officially finish
import time
time.sleep(10)

single_result = pipeline.infer(dfImage)
display(single_result)
display(pipeline.steps())

 ok


,time,in.tensor,out.boxes,out.classes,out.confidences,check_failures
0,2023-12-08 18:16:17.916,"[0.9372549057, 0.9529411793, 0.9490196109, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9450980425, 0.9490196109, 0.9490196109, 0.9529411793, 0.9529411793, 0.9490196109, 0.9607843161, 0.9686274529, 0.9647058845, 0.9686274529, 0.9647058845, 0.9568627477, 0.9607843161, 0.9647058845, 0.9647058845, 0.9607843161, 0.9647058845, 0.9725490212, 0.9568627477, 0.9607843161, 0.9176470637, 0.9568627477, 0.9176470637, 0.8784313798, 0.8941176534, 0.8431372643, 0.8784313798, 0.8627451062, 0.850980401, 0.9254902005, 0.8470588326, 0.9686274529, 0.8941176534, 0.8196078539, 0.850980401, 0.9294117689, 0.8666666746, 0.8784313798, 0.8666666746, 0.9647058845, 0.9764705896, 0.980392158, 0.9764705896, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.9725490212, 0.980392158, 0.8941176534, 0.4823529422, 0.4627451003, 0.4313725531, 0.270588249, 0.2588235438, 0.2941176593, 0.3450980484, 0.3686274588, 0.4117647111, 0.4549019635, 0.4862745106, 0.5254902244, 0.5607843399, 0.6039215922, 0.6470588446, 0.6862745285, 0.721568644, 0.7450980544, 0.7490196228, 0.7882353067, 0.8666666746, 0.980392158, 0.9882352948, 0.9686274529, 0.9647058845, 0.9686274529, 0.9725490212, 0.9647058845, 0.9607843161, 0.9607843161, 0.9607843161, 0.9607843161, ...]","[2.1511102, 193.98323, 76.26535, 475.40292, 610.82245, 98.606316, 639.8868, 232.27054, 544.2867, 98.726524, 581.28845, 230.20497, 454.99344, 113.08567, 484.78464, 210.1282, 502.58884, 331.87665, 551.2269, 476.49182, 538.54254, 292.1205, 587.46545, 468.1288, 578.5417, 99.70756, 617.2247, 233.57082, 548.552, 191.84564, 577.30585, 238.47737, 459.83328, 344.29712, 505.42633, 456.7118, 483.47168, 110.56585, 514.0936, 205.00156, 262.1222, 190.36658, 323.4903, 405.20584, 511.6675, 104.53834, 547.01715, 228.23663, 75.39197, 205.62312, 168.49893, 453.44086, 362.50656, 173.16858, 398.66956, 371.8243, 490.42468, 337.627, 534.1234, 461.0242, 351.3856, 169.14897, 390.7583, 244.06992, 525.19824, 291.73895, 570.5553, 417.6439, 563.4224, 285.3889, 609.30853, 452.25943, 425.57935, 366.24915, 480.63535, 474.54, 154.538, 198.0377, 227.64284, 439.84412, 597.02893, 273.60458, 637.2067, 439.03214, 473.88763, 293.41992, 519.7537, 349.2304, 262.77597, 192.03581, 313.3096, 258.3466, 521.1493, 152.89026, 534.8596, 246.52365, 389.89633, 178.07867, 431.87555, 360.59323, ...]","[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 86, 82, 44, 44, 44, 44, 44, 44, 84, 84, 44, 44, 44, 44, 86, 84, 44, 44, 44, 44, 44, 84, 44, 44, 84, 44, 44, 44, 44, 51, 44, 44, 44, 44, 44, 44, 44, 44, 44, 82, 44, 44, 44, 44, 44, 86, 44, 44, 1, 84, 44, 44, 44, 44, 84, 47, 47, 84, 14, 44, 44, 53, 84, 47, 47, 44, 84, 44, 44, 82, 44, 44, 44]","[0.9965358, 0.9883404, 0.9700247, 0.9696426, 0.96478045, 0.96037567, 0.9542889, 0.9467539, 0.946524, 0.94484967, 0.93611854, 0.91653466, 0.9133634, 0.8874814, 0.84405905, 0.825526, 0.82326967, 0.81740034, 0.7956525, 0.78669065, 0.7731486, 0.75193685, 0.7360918, 0.7009186, 0.6932351, 0.65077204, 0.63243586, 0.57877576, 0.5023476, 0.50163734, 0.44628552, 0.42804396, 0.4253787, 0.39086252, 0.36836442, 0.3473236, 0.32950658, 0.3105372, 0.29076362, 0.28558296, 0.26680034, 0.26302803, 0.25444376, 0.24568668, 0.2353662, 0.23321979, 0.22612995, 0.22483191, 0.22332378, 0.21442991, 0.20122288, 0.19754867, 0.19439234, 0.19083925, 0.1871393, 0.17646024, 0.16628945, 0.16326219, 0.14825206, 0.13694529, 0.12920643, 0.12815322, 0.122357555, 0.121289656, 0.116281696, 0.11498632, 0.111848116, 0.11016138, 0.1095062, 0.1039151, 0.10385688, 0.097573474, 0.09632071, 0.09557622, 0.091599144, 0.09062039, 0.08262358, 0.08223499, 0.07993951, 0.07989185, 0.078758545, 0.078201495, 0.07737936, 0.07690251, 0.07593444, 0.07503418, 0.07482597, 0.06898

[{'ModelInference': {'models': [{'name': 'resnet50', 'version': 'd8712e05-c1a4-4b15-b313-dacb570f4379', 'sha': '43326e50af639105c81372346fb9ddf453fea0fe46648b2053c375360d9c1647'}]}}]

### Cleaning up.

At this point, if you are not continuing on to the next notebook, undeploy your pipeline(s) to give the resources back to the environment.

In [19]:
## blank space to undeploy the pipelines

pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,cv-retail
created,2023-12-08 18:10:33.817785+00:00
last_updated,2023-12-08 18:16:07.409747+00:00
deployed,False
arch,None
tags,
versions,"01ce3a4c-eb4c-4a9d-acea-d4b8fde02d46, e791bb6f-1f0c-49be-8f27-3c987cdee4ab, 43ef6552-12c1-4d4c-a8d0-58a0209a721f, 7ce5dd31-5851-492b-ba36-80e4adf8ef04, 0a4ce878-3f59-4134-b32f-1a6946db9775, e7149fb9-8253-4e87-b9a6-a220c8df5639, c6abcdaa-047b-4eef-8f83-eb2dcc8782d1"
steps,mobilenet
published,False


## Congratulations!

You have now 
* successfully trained new challenger models for the house price prediction problem
* compared your models using an A/B test
* compared your models using a shadow deployment
* replaced your old model for a new one in the house price prediction pipeline

In the next notebook, you will learn how to monitor your production pipeline for "anomalous" or out-of-range behavior.